In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
import pprint
import pickle
import glob
import random
from pathlib import Path

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import (
    look_at_view_transform
)
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np

from utils import utils
import deformation.losses as def_losses
from deformation.semantic_discriminator_loss import SemanticDiscriminatorLoss 
from semantic_discriminator_trainer import train
from deformation.semantic_discriminator_dataset import SemanticDiscriminatorDataset
from deformation.semantic_discriminator_net import SemanticDiscriminatorNetwork
from adversarial.datasets import GenerationDataset, ShapenetRendersDataset


In [4]:
gpu_num = 0
device = torch.device("cuda:"+str(gpu_num))
cfg_path = "configs/default.yaml"
cfg = utils.load_config(cfg_path, "configs/default.yaml")

/home/svcl-oowl/brandon/research/sil_consistent_at_inference/utils/utils.py:52: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg_special = yaml.load(f)
/home/svcl-oowl/brandon/research/sil_consistent_at_inference/utils/utils.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(f)


In [5]:
generation_dataset = GenerationDataset(cfg, device)
generation_loader = torch.utils.data.DataLoader(
    generation_dataset, batch_size=6, num_workers=0, shuffle=True)



In [21]:
for gen_batch in generation_loader:
    print(gen_batch["pose"].shape)
    print(gen_batch["image"].shape)
    print(gen_batch["mesh_verts"].shape)
    break

torch.Size([6, 3])
torch.Size([6, 3, 224, 224])
torch.Size([6, 1502, 3])


In [14]:
shapenet_renders_dataset = ShapenetRendersDataset(cfg)
shapenet_renders_loader = torch.utils.data.DataLoader(shapenet_renders_dataset, batch_size=6, num_workers=0, shuffle=True)

In [20]:
generation_iter = iter(generation_loader)
shapenet_renders_iter = iter(shapenet_renders_loader)
for batch_idx in range(3):
    gen_batch = next(generation_iter)
    print(gen_batch["pose"])
    print(gen_batch["pose"].shape)
    print(gen_batch["image"].shape)
    print(gen_batch["mesh_verts"].shape)
    real_render_batch = next(shapenet_renders_iter)
    print(real_render_batch.shape)
    break

tensor([[  1.1410,  18.9474, 341.0526],
        [  1.2051,   0.0000, 284.2105],
        [  1.0769,   9.4737, 113.6842],
        [  1.0128, 123.1579,  94.7368],
        [  0.8846,  94.7368, 132.6316],
        [  1.0769, 142.1053, 303.1579]])
torch.Size([6, 3])
torch.Size([6, 3, 224, 224])
torch.Size([6, 1502, 3])
torch.Size([6, 3, 224, 224])


In [ ]:
tensor([[  1.2692, 142.1053, 360.0000],
        [  1.0769, 142.1053,   0.0000],
        [  1.0128, 132.6316, 227.3684],
        [  1.2051, 142.1053, 322.1053],
        [  1.0769, 151.5789,  56.8421],
        [  1.2051,   9.4737, 322.1053]])